In [ ]:
#OBS: 
# Precisa do azure configurado
# Precisa do purview configurado no azure

In [ ]:
import requests
import json
import pandas as pd

from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient, AtlasEntity, AtlasProcess, TypeCategory
from pyapacheatlas.core.typedef import *


In [ ]:
def azuread_auth(tenant_id: str, client_id: str, client_secret: str, resource_url: str):
    
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    payload= f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}&resource={resource_url}'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    access_token = json.loads(response.text)['access_token']
    
    return access_token

In [ ]:
def purview_auth(tenant_id: str, client_id: str, client_secret: str, data_catalog_name: str):
  
    oauth2 = ServicePrincipalAuthentication(
        tenant_id = tenant_id,
        client_id = client_id,
        client_secret = client_secret
    )
    client = PurviewClient(
        account_name = data_catalog_name,
        authentication = oauth2
    )
    return client

In [ ]:
# Abrir conexão com o datalake
spark.conf.set(
  "", # Endpoint do datalake
  "" # Acess Key
)

In [ ]:
#criar json com as chaves de acesso do azure e purview

# Recuperar caminho até o json de parâmetros da Autentificação do Purview
path_params = dbutils.fs.ls('''path do json no azure''')

# Recuperar dados do json de parâmetros da Autentificação do Purview
parameters = spark.read.format("json").load(path_params)

client_id = parameters.select('client_id').collect()[0][0]
client_secret = parameters.select('client_secret').collect()[0][0]
data_catalog_name = parameters.select('data_catalog_name').collect()[0][0]
resource_url = parameters.select('resource_url').collect()[0][0]
tenant_id = parameters.select('tenant_id').collect()[0][0]

# Recuperar objetos de autenticação
azuread_access_token = azuread_auth(tenant_id, client_id, client_secret, resource_url)
purview_client = purview_auth(tenant_id, client_id, client_secret, data_catalog_name)

In [ ]:
list_response_values = []
def requestId(endpoint: str, url: str, response: any, payload: any):
    endpoint = endpoint
    url = url
    payload = payload
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {azuread_access_token}'
    }
    response = response
    list_response_values.append(response.text)
    
    if(response.status_code != 200):
      print("Status:",response.status_code, "Erro no código")
    else:
      print("Status:",response.status_code, "Ok", list_response_values)
    
    return list_response_values

In [ ]:
endpoint = "https://{data_catalog_name}.purview.azure.com/"
url = f"{endpoint}/catalog/api/search/query?api-version=2021-05-01-preview"
method = "POST"
payload = json.dumps({
        "orderby":["name"],
        "limit": 1000,
        "keywords": None,
        "filter": {
            "classification": "",
            "includeSubClassifications": True,
            "collectionId": ""
        }
    })
response = requests.request("POST", url, headers=headers, data=payload)

requestId(endpoint, url, response, payload)

In [ ]:
df_value = pd.DataFrame(json.loads(list_response_values[0])['value'])
#df_value

In [ ]:
df_id = df_value['id']
#df_id

In [ ]:
list_response = []
for id_ in df_id:
    endpoint = "https://{data_catalog_name}.purview.azure.com/"
    url = f"{endpoint}/catalog/api/atlas/v2/entity/bulk?excludeRelationshipTypes=dataset_process_inputs&excludeRelationshipTypes=process_parent&excludeRelationshipTypes=direct_lineage_dataset_dataset&guid={id_}&includeTermsInMinExtInfo=false&minExtInfo=false&ignoreRelationships=false"
    payload = azuread_access_token
    
    response = requests.request("GET", url, headers=headers, data=payload)
    list_response.append(response.text)
    requestId(endpoint, url, response, payload)

In [ ]:
df_list_res_referredEntities = pd.DataFrame()

for x in list_response:
    df_list = pd.DataFrame(json.loads(x)['referredEntities']).loc[['attributes','classifications'],:].transpose()
    
    df_list_res_referredEntities = pd.concat([df_list_res_referredEntities, df_list])


df_list_res_referredEntities.reset_index(inplace=True)
df_list_res_referredEntities = df_list_res_referredEntities.rename(columns = {'index':'IDs'})

#df_list_res_referredEntities

In [ ]:
df_columns = df_list_res_referredEntities

df_columns['classificação'] = ''

for i in range(df_columns.shape[0]):
    df_columns.loc[i,'attributes'] = df_columns.loc[i,'attributes']['name']
    
    if pd.isna(df_columns.loc[:,'classifications'].iloc[i]):
        df_columns.loc[:,'classificação'].iloc[i] = 'Sem classificação' 
    else:
        df_columns.loc[:,'classificação'].iloc[i] = df_columns.classifications.iloc[i][0]['typeName']

#df_columns

In [ ]:
df_col = df_columns.loc[:, ['IDs','attributes', 'classificação']]
#df_col

In [ ]:
df_tables = df_value.loc[:, ['id','name', 'classification']]
#df_tables


In [ ]:
df_tables['classificação2'] = ''

for counter, list_ in enumerate(df_tables['classification']):
    if('sua classificação' in list_ and 'sua classificação' in list_):
        df_tables.loc[counter, 'classificação2'] = 'sua classificação'

#df_tables

In [ ]:
df_tab = df_tables.loc[:, ['id', 'name', 'classificação2']]
#df_tab

In [ ]:
# 1 - criando dataframe
# 2 - trazendo id, nome e classificação das tabelas
# 3 - criando nova coluna com o id das colunas
# 4 - igualando o id da tabela com a coluna para fazer o merge

df_tab.loc[:,'Id_simplificado'] = ''
for count, item in enumerate(df_tab.id):
    df_tab.loc[:,'Id_simplificado'].iloc[count] = item[:33]

df_col.loc[:,'Id_simplificado'] = ''    
for count, item in enumerate(df_col.IDs):
    df_col.loc[:,'Id_simplificado'].iloc[count] = item[:33]
    
    
#print(df_col.head())
#df_tab.head()

In [ ]:
table_merge = df_tab.merge(df_col, left_on='Id_simplificado', right_on='Id_simplificado')
#table_merge

In [ ]:
table_final = table_merge[['name', 'classificação2', 'attributes', 'classificação']].rename(columns={'name':'Tabela', 'classificação2':'Classificação', 'attributes':'Colunas', 'classificação':'Classificação'})

table_final

In [ ]:
def export_file(dataframe, ext_target, target, new_name):
  df_res = dataframe
  df_res.to_csv('/dbfs/tmp/purview/'+new_name+'.'+ext_target, sep=',', index=False)
  dbutils.fs.mv("/tmp/purview/"+new_name+'.'+ext_target, target+'/'+new_name+'.'+ext_target)
  print ('Novo arquivo {} salvo em {}'.format(new_name, target))


In [ ]:
dataframe = table_final
ext_target="csv"
path_raw = 'path do azure'
new_name = f"nome do arquivo csv"
export_file(dataframe, ext_target, path_raw, new_name)